In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# This Colab requires TF 2.5.
!pip install -U "tensorflow>=2.5"

In [ ]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt
import re
import pandas as pd
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [ ]:
# @title Run this!!

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


ALL_MODELS = {
'CenterNet HourGlass104 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512/1',
'CenterNet HourGlass104 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512_kpts/1',
'CenterNet HourGlass104 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024/1',
'CenterNet HourGlass104 Keypoints 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024_kpts/1',
'CenterNet Resnet50 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512/1',
'CenterNet Resnet50 V1 FPN Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512_kpts/1',
'CenterNet Resnet101 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet101v1_fpn_512x512/1',
'CenterNet Resnet50 V2 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512/1',
'CenterNet Resnet50 V2 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512_kpts/1',
'EfficientDet D0 512x512' : 'https://tfhub.dev/tensorflow/efficientdet/d0/1',
'EfficientDet D1 640x640' : 'https://tfhub.dev/tensorflow/efficientdet/d1/1',
'EfficientDet D2 768x768' : 'https://tfhub.dev/tensorflow/efficientdet/d2/1',
'EfficientDet D3 896x896' : 'https://tfhub.dev/tensorflow/efficientdet/d3/1',
'EfficientDet D4 1024x1024' : 'https://tfhub.dev/tensorflow/efficientdet/d4/1',
'EfficientDet D5 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d5/1',
'EfficientDet D6 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d6/1',
'EfficientDet D7 1536x1536' : 'https://tfhub.dev/tensorflow/efficientdet/d7/1',
'SSD MobileNet v2 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2',
'SSD MobileNet V1 FPN 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v1/fpn_640x640/1',
'SSD MobileNet V2 FPNLite 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1',
'SSD MobileNet V2 FPNLite 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_640x640/1',
'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_640x640/1',
'SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_1024x1024/1',
'SSD ResNet101 V1 FPN 640x640 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_640x640/1',
'SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_1024x1024/1',
'SSD ResNet152 V1 FPN 640x640 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_640x640/1',
'SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_1024x1024/1',
'Faster R-CNN ResNet50 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1',
'Faster R-CNN ResNet50 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_1024x1024/1',
'Faster R-CNN ResNet50 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_800x1333/1',
'Faster R-CNN ResNet101 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_640x640/1',
'Faster R-CNN ResNet101 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_1024x1024/1',
'Faster R-CNN ResNet101 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_800x1333/1',
'Faster R-CNN ResNet152 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_640x640/1',
'Faster R-CNN ResNet152 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_1024x1024/1',
'Faster R-CNN ResNet152 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_800x1333/1',
'Faster R-CNN Inception ResNet V2 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1',
'Faster R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1',
'Mask R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1'
}

COCO17_HUMAN_POSE_KEYPOINTS = [(0, 1),
 (0, 2),
 (1, 3),
 (2, 4),
 (0, 5),
 (0, 6),
 (5, 7),
 (7, 9),
 (6, 8),
 (8, 10),
 (5, 6),
 (5, 11),
 (6, 12),
 (11, 12),
 (11, 13),
 (13, 15),
 (12, 14),
 (14, 16)]

In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Installing the Object Detection API

In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .
pip install numpy --upgrade


Now we can import the dependencies we will need later

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline

In [ ]:
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
#@title Model Selection { display-mode: "form", run: "auto" }
model_display_name = 'Faster R-CNN Inception ResNet V2 1024x1024' # @param ['CenterNet HourGlass104 512x512','CenterNet HourGlass104 Keypoints 512x512','CenterNet HourGlass104 1024x1024','CenterNet HourGlass104 Keypoints 1024x1024','CenterNet Resnet50 V1 FPN 512x512','CenterNet Resnet50 V1 FPN Keypoints 512x512','CenterNet Resnet101 V1 FPN 512x512','CenterNet Resnet50 V2 512x512','CenterNet Resnet50 V2 Keypoints 512x512','EfficientDet D0 512x512','EfficientDet D1 640x640','EfficientDet D2 768x768','EfficientDet D3 896x896','EfficientDet D4 1024x1024','EfficientDet D5 1280x1280','EfficientDet D6 1280x1280','EfficientDet D7 1536x1536','SSD MobileNet v2 320x320','SSD MobileNet V1 FPN 640x640','SSD MobileNet V2 FPNLite 320x320','SSD MobileNet V2 FPNLite 640x640','SSD ResNet50 V1 FPN 640x640 (RetinaNet50)','SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)','SSD ResNet101 V1 FPN 640x640 (RetinaNet101)','SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)','SSD ResNet152 V1 FPN 640x640 (RetinaNet152)','SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)','Faster R-CNN ResNet50 V1 640x640','Faster R-CNN ResNet50 V1 1024x1024','Faster R-CNN ResNet50 V1 800x1333','Faster R-CNN ResNet101 V1 640x640','Faster R-CNN ResNet101 V1 1024x1024','Faster R-CNN ResNet101 V1 800x1333','Faster R-CNN ResNet152 V1 640x640','Faster R-CNN ResNet152 V1 1024x1024','Faster R-CNN ResNet152 V1 800x1333','Faster R-CNN Inception ResNet V2 640x640','Faster R-CNN Inception ResNet V2 1024x1024','Mask R-CNN Inception ResNet V2 1024x1024']
model_handle = ALL_MODELS[model_display_name]


In [ ]:
print('loading model...')
hub_model = hub.load(model_handle)
print('model loaded!')

In [ ]:
def filter_weird_small_box(output_d):
    '''
    Steps:
    1. calculate width and depth of boxes using normalized coordinates
    2. get a threshold for area, by referencing to roughly the largest-valid-detected boxes (here used: 11_March_4706_6.jpg, a truck with half yaxis tall)
    3. get a threshold for width (d1), coz it couldn't be too long also
    4.
    '''
    idx_to_del = []
    for k in range(len(output_d['detection_boxes'][0])):
        lst_1 = output_d['detection_boxes'][0][k]
        p1,p2,p3 = [lst_1[0],lst_1[1]],[lst_1[0],lst_1[3]],[lst_1[2],lst_1[3]]
        # d1,d2 = width,depth
        d1,d2 = ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5 , ((p2[0]-p3[0])**2 + (p2[1]-p3[1])**2)**0.5
        area = d1*d2
        scaling_factor = 1.3
        if d1 < 0.40 or d2 < 0.10: # this line is to filter the extreme wide and extreme tall boxes
            idx_to_del.append(k)
    idx_to_del = sorted(set(idx_to_del), reverse = True)
    for i in idx_to_del:
      output_d['detection_boxes'] = np.concatenate([output_d['detection_boxes'][:, :i], output_d['detection_boxes'][:, i+1:]], axis = 1)
      output_d['detection_classes'] = np.concatenate([output_d['detection_classes'][:, :i], output_d['detection_classes'][:, i+1:]], axis = 1)
      output_d['detection_scores'] = np.concatenate([output_d['detection_scores'][:, :i], output_d['detection_scores'][:, i+1:]], axis = 1)
    output_d['num_detections'] -= len(idx_to_del)
    return output_d


def filter_doublecount_overlap(output_d, overlap_thres):
    idx_to_del = []
    idx_done = []

    valid_classes = [3.0, 6.0, 7.0, 8.0]

    for k in range(len(output_d['detection_boxes'][0])):
        class_k = output_d['detection_classes'][0][k]
        lst_1 = output_d['detection_boxes'][0][k]
        idx_done.append(k)

        for m in range(len(output_d['detection_boxes'][0])):
            if k != m:
                class_m = output_d['detection_classes'][0][m]

                if class_k in valid_classes and class_m in valid_classes:
                    lst_2 = output_d['detection_boxes'][0][m]
                    lst_x, lst_y = [[lst_1[1],'1'], [lst_1[3],'1'],[lst_2[1],'2'], [lst_2[3],'2']],[[lst_1[0],'1'], [lst_1[2],'1'],[lst_2[0],'2'], [lst_2[2],'2']]
                    lst_x.sort(key=lambda x:x[0])
                    lst_y.sort(key=lambda x:x[0])
                    l_x, l_y = lst_1[3] - lst_1[1], lst_1[2] - lst_1[0]

                    is_intersect = True
                    if lst_x[0][1] == lst_x[1][1] or lst_y[0][1] == lst_y[1][1]:
                        is_intersect = False

                    exceed_x, exceed_y = False, False
                    if lst_x[2][0] - lst_x[1][0] > overlap_thres * l_x and is_intersect:
                        exceed_x = True
                    if lst_y[2][0] - lst_y[1][0] > overlap_thres * l_y and is_intersect:
                        exceed_y = True

                    exceed_area = False
                    area_overlap = (lst_x[2][0] - lst_x[1][0]) * (lst_y[2][0] - lst_y[1][0])
                    area_box = l_x * l_y
                    if area_overlap > overlap_thres * area_box and is_intersect:
                        exceed_area = True

                    is_doublecount = (exceed_x and exceed_y) or (exceed_x or exceed_area) and is_intersect
                    if is_doublecount and m not in idx_done and m not in idx_to_del:
                        idx_to_del.append(m)

    idx_to_del = sorted(set(idx_to_del), reverse=True)

    for i in idx_to_del:
      output_d['detection_boxes'] = np.concatenate([output_d['detection_boxes'][:, :i], output_d['detection_boxes'][:, i+1:]], axis = 1)
      output_d['detection_classes'] = np.concatenate([output_d['detection_classes'][:, :i], output_d['detection_classes'][:, i+1:]], axis = 1)
      output_d['detection_scores'] = np.concatenate([output_d['detection_scores'][:, :i], output_d['detection_scores'][:, i+1:]], axis = 1)
    output_d['num_detections'] -= len(idx_to_del)
    return output_d

In [ ]:
def display_box_for_image(image_np, i, day):
  ccount = 0
  count_car = 0
  count_motorcycle = 0
  count_bus = 0
  count_train = 0
  count_truck = 0
  bounding_box_coordinates = []
  # running inference
  results = hub_model(image_np)
  # print(results['detection_scores'][0])
  # print(results['detection_classes'][0])

# different object detection models have additional results
# all of them are explained in the documentation
  result = {key:value.numpy() for key,value in results.items()}

  label_id_offset = 0
  image_np_with_detections = image_np.copy()
# Use keypoints if available in detections
  keypoints, keypoint_scores = None, None
  if 'detection_keypoints' in result:
    keypoints = result['detection_keypoints'][0]
    keypoint_scores = result['detection_keypoint_scores'][0]
  #filter_weird_small_box(result)
  filter_doublecount_overlap(result, 0.3)

  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=5,
      min_score_thresh=.30,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores,
      keypoint_edges=COCO17_HUMAN_POSE_KEYPOINTS)
  plt.figure(figsize=(24,32))
  # plt.imshow(image_np_with_detections[0])
  # plt.show()
  im = Image.fromarray(image_np_with_detections[0])
  im.save("/content/drive/MyDrive/A*STAR Traffic Project/20230202/"+str(day)+"/Bounding Boxes__" + str(i) + "_.jpg")
  bboxes = []

  for i in range(len(result['detection_classes'][0])):
    if result['detection_classes'][0][i] == 3 and result['detection_scores'][0][i] > 0.30:
      bboxes.append(("car", result['detection_boxes'][0][i]))
      count_car+= 1

  for i in range(len(result['detection_classes'][0])):
    if result['detection_classes'][0][i] == 4 and result['detection_scores'][0][i] > 0.30:
      bboxes.append(("motorcycle", result['detection_boxes'][0][i]))
      count_motorcycle+= 1

  for i in range(len(result['detection_classes'][0])):
    if result['detection_classes'][0][i] == 6 and result['detection_scores'][0][i] > 0.30:
      bboxes.append(("bus", result['detection_boxes'][0][i]))
      count_bus+= 1

  for i in range(len(result['detection_classes'][0])):
    if result['detection_classes'][0][i] == 7 and result['detection_scores'][0][i] > 0.30:
      bboxes.append(("train", result['detection_boxes'][0][i]))
      count_train+= 1

  for i in range(len(result['detection_classes'][0])):
    if result['detection_classes'][0][i] == 8 and result['detection_scores'][0][i] > 0.30:
      bboxes.append(("truck", result['detection_boxes'][0][i]))
      count_truck+= 1

  final_box = []
  for box in bboxes:
    ymin, xmin, ymax, xmax = box[1]
    final_box.append((box[0], [xmin * 1920, xmax * 1920, ymin * 1080, ymax * 1080]))
  ccount = count_car + count_motorcycle + count_bus + count_train + count_truck

  return ccount, count_car, count_motorcycle, count_bus, count_train, count_truck, final_box

In [ ]:
def get_image_text_file(i, day):
  x = load_image_into_numpy_array(IMAGES_FOR_TEST['Cropped_Image' + str(i)])
  ccount, count_car, count_motorcycle, count_bus, count_train, count_truck, bounding_box_coordinates = display_box_for_image(x, i, day)
  frame_timestamp = 10 * (i-1)
  row = [str(day), frame_timestamp, count_car, count_motorcycle, count_bus, count_train, count_truck, ccount]
  df.loc[i-1] = row

In [ ]:
#the number of days in the list of
day_list =['GX240029', 'GX250029']

In [ ]:
!pip3 install Pillow==9.5.0

In [ ]:
for day in day_list:
  # Specify the directory containing the images
  image_directory = '/content/drive/MyDrive/A*STAR Traffic Project/20230202/' + str(day)
  # Get the list of files in the directory
  image_files = os.listdir(image_directory)
  # Get the list of files in the directory
  image_files = os.listdir(image_directory)

  # Filter the list to include only image files
  image_files = [file for file in image_files if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]

  # Create dictionaries to store the image paths
  images_dict = {}
  cropped_images_dict = {}

  # Iterate over the image files and add them to the dictionaries
  for file in image_files:
      if 'Cropped_Image' in file:
          key = file.replace('.jpg', '').replace('.jpeg', '').replace('.png', '').replace('.gif', '')
          cropped_images_dict[key] = os.path.join(image_directory, file)
      elif 'Image' in file:
          key = file.replace('.jpg', '').replace('.jpeg', '').replace('.png', '').replace('.gif', '')
          images_dict[key] = os.path.join(image_directory, file)

  # Combine both dictionaries
  IMAGES_FOR_TEST = {**images_dict, **cropped_images_dict}


  df = pd.DataFrame(columns=["video_filename", "frame / time-stamp", "model-cars", "model-motorcycles", "model-bus", "model-train", "model-truck", "model-vehicles"])

  largest_number = max(int(re.search(r'\d+', key).group()) for key in IMAGES_FOR_TEST.keys())
  for i in range(1, largest_number+1):
    get_image_text_file(i, day)

  excel_file_path = "/content/drive/MyDrive/A*STAR Traffic Project/20230202/" + str(day) + ".xlsx"
  df.to_excel(excel_file_path, sheet_name="Data", index=False)